In [ ]:
import pandas as pd
import mygene
import os,sys, gc
sys.path.append("/home/fvalle/phd/master_thesis/hsbm/")

In [ ]:
client = mygene.get_client('gene')

In [ ]:
df = pd.read_csv("mainTable_all.csv", index_col=[0]).applymap(lambda x: pd.np.log2(x + 1))

In [ ]:
df_genes = pd.read_csv("HDE_Breast.txt", header=None)
df_conversion = pd.read_csv("https://www.genenames.org/cgi-bin/download/custom?col=gd_app_sym&col=gd_pub_ensembl_id&status=Approved&status=Entry%20Withdrawn&hgnc_dbtag=on&order_by=gd_app_sym_sort&format=text&submit=submit", sep="\t", index_col=0)
HVE_genes_ensg = []
for g in pd.read_csv("HDE_Breast.txt", header=None).values.ravel():
    if g in df_conversion.index:
        HVE_genes_ensg.append(df_conversion.at[g, 'Ensembl gene ID'])
    else:
        q = client.query(g, fields='symbol,ensembl')
        if len(q['hits'])>0:
            if  'ensembl' in q['hits'][0].keys():
                if len(q['hits'][0]['ensembl'])==0:
                    if 'gene' in q['hits'][0]['ensembl'].keys():
                        if 'ENSG' in q['hits'][0]['ensembl']['gene']:
                            HVE_genes_ensg.append(q['hits'][0]['ensembl']['gene'])
                        else:
                            print("NO ensg for %s"%g, q)
                    else:
                        print(g, q)
                else:
                    print(g, q)
        else:
            print(g, q)
pd.DataFrame(data=[g for g in HVE_genes_ensg if len(str(g))==15]).dropna().to_csv("HVE_genes_ensg.txt", index=False, header=None)

In [ ]:
HVE_genes_ensg = pd.read_csv("HVE_genes_ensg.txt", header=None).values.ravel()

In [ ]:
df = df[df.index.isin(HVE_genes_ensg[:2000])]

In [ ]:
gc.collect()

In [ ]:
from sbmtm import sbmtm

In [ ]:
hsbm = sbmtm()
#hsbm.load_graph("graph.xml.gz")
hsbm.make_graph_from_BoW_df(df.round().astype(int), counts=False)
#hsbm.save_graph("graph.xml.gz")

In [ ]:
hsbm.fit()

In [ ]:
hsbm.save_data()

In [ ]:
hsbm.dump_model()

In [ ]:
hsbm.clusters()

In [ ]:
import graph_tool as gt

In [ ]:
gt.spectral.adjacency(hsbm.g).toarray()

In [ ]:
hsbm.g.properties

In [ ]:
gc.collect()

In [ ]:
df = pd.DataFrame(data=[[1,2,3],[4,5,6]], index=['w1', 'w2'], columns = ['s1', 's2', 's3'])

In [ ]:
del hsbm